# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

**Реализация**

In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\imphi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# функция-алгоритм Леска
def lesk( word, sentence ):
    
    lod = dict() #словарь в котором хранится индекс значения и кол-во пересечений
    maxoverlap = 0 #кол-во пересечений
    
    for indx, synset in enumerate(wn.synsets(word)):
        for d_word in synset.definition().split():
            for c_word in sentence:
                if c_word == d_word:
                    
                    maxoverlap += 1
                    
        lod[indx] = maxoverlap
        maxoverlap = 0
    
    global bestsense
    bestsense = max(lod, key=lod.get) #достаю key-индекс синсета с наибольшим value-пересечением 
    
    return bestsense

In [3]:
# на вход НУЖНО подавать элементы результата работы функции get_words_in_context, однако в действительности мы не писали её
# здесь используется только дефолтный для данного ноутбука пример, где контекст совпадает с одним из определений
lesk('day', 'some point or period in time'.split())

1

Посмотрим на определения самостоятельно, чтобы удостовериться в том, что алгоритм не ошибся.

In [4]:
word = 'day'
for synset in wn.synsets(word):
    print(word + ' - ' + synset.definition())

day - time for Earth to make a complete rotation on its axis
day - some point or period in time
day - a day assigned to a particular purpose or observance
day - the time after sunrise and before sunset while it is light outside
day - the recurring hours when you are not sleeping (especially those when you are working)
day - an era of existence or influence
day - the period of time taken by a particular planet (e.g. Mars) to make a complete rotation on its axis
day - the time for one complete rotation of the earth relative to a particular star, about 4 minutes shorter than a mean solar day
day - a period of opportunity
day - United States writer best known for his autobiographical works (1874-1935)


**Проверим насколько хорошо работает такой метод на датасете из семинара**

Нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным (посчитать процент правильных предсказаний - accuracy).

In [5]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
# чтобы считалось быстрее, был взят не весь корпус
for sent in corpus[:10000]:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [6]:
window = 5 # отвечает за размер контекста - здесь берутся 10 ближайшиз слов (5 слева и 5 справа)
rights = 0 # кол-во правильных предсказаний
count = 0 # кол-во многозначных слов

for sent_wsd in corpus_wsd:
     sentence = [w_wsd[2] for w_wsd in sent_wsd] # реконструкция предложения в виде списка
     for word_wsd in sent_wsd:
        # работаем только с многозначными словами
        if word_wsd[0] != '':
            count += 1
            l = sent_wsd.index(word_wsd) # показывает позицию слова в предложении
            context = sentence[max(0, l-window):l] + sentence[l+1:l+window+1]
            lesk(word_wsd[1], context)
            mypred = wn.synsets(word_wsd[1])[bestsense]
            wsdpred = wn.lemma_from_key(word_wsd[0]).synset()
            if mypred == wsdpred:
                rights += 1

accuracy = rights/count

In [7]:
print('Общее число многозначных слов в корпусе: {}'.format(count))
print('Правильные предсказания: {:.2%}'.format(accuracy))

Общее число многозначных слов в корпусе: 93950
Правильные предсказания: 38.89%
